In [1]:
include("Axis_Aligned_Box.jl");
type Mondrian_Node
    parent::Nullable{Mondrian_Node}
    left::Nullable{Mondrian_Node}
    right::Nullable{Mondrian_Node}
    τ::Float64
    node_type::Array{Bool,1}        # node,leaf,root
    δ::Nullable{Int}                # split dimension
    ζ::Nullable{Float64}            # split position
    Θ::Nullable{Axis_Aligned_Box}   # data boxes B
    tab::Array{Int}
    c::Array{Int}
    Gₚ::Array{Float64}
end

function Mondrian_Node()
    N = Mondrian_Node(Nullable{Mondrian_Node}(),
                      Nullable{Mondrian_Node}(),
                      Nullable{Mondrian_Node}(),
                      0,[true,false,false],
                      Nullable{Int}(),
                      Nullable{Float64}(),
                      Nullable{Axis_Aligned_Box}(),
                      Array{Int}(),
                      Array{Int}(),
                      Array{Float64}())
    return N
end

function Mondrian_Node(τ::Float64)
    N = Mondrian_Node(Nullable{Mondrian_Node}(),
                      Nullable{Mondrian_Node}(),
                      Nullable{Mondrian_Node}(),
                      τ,[true,false,false],
                      Nullable{Int}(),
                      Nullable{Float64}(),
                      Nullable{Axis_Aligned_Box}(),
                      Array{Int}(),
                      Array{Int}(),
                      Array{Float64}())
    return N
end

function Mondrian_Node(τ::Float64, node_type::Array{Bool,1})
    N = Mondrian_Node(Nullable{Mondrian_Node}(),
                      Nullable{Mondrian_Node}(),
                      Nullable{Mondrian_Node}(),
                      τ,
                      node_type,
                      Nullable{Int}(),
                      Nullable{Float64}(),
                      Nullable{Axis_Aligned_Box}(),
                      Array{Int}(),
                      Array{Int}(),
                      Array{Float64}())
    return N
end

type Mondrian_Tree
    root::Nullable{Mondrian_Node}
    leaves::Array{Mondrian_Node,1}
end

function Mondrian_Tree()
    T = Mondrian_Tree(Nullable{Mondrian_Node}(),Array{Mondrian_Node,1}())
end

function Mondrian_Tree(N::Mondrian_Node)
    T = Mondrian_Tree(N,Array{Mondrian_Node,1}())
end

Mondrian_Tree

In [174]:
using Distributions

function Sample_Mondrian_Tree(λ,D)
    δ = []
    ζ = []
    n = 1:size(D,1)
    e = Mondrian_Node(0.0,[false,false,true])
    T = Mondrian_Tree(e)
    Θ = Axis_Aligned_Box(get_intervals(D[1]))
    e.Θ = Θ
    e.tab = zeros(size(unique(D[2]),1))
    e.c = zeros(size(unique(D[2]),1))
    e.Gₚ = zeros(size(unique(D[2]),1))
    Sample_Mondrian_Block(e, Θ, λ, δ, ζ, T)
    return T
end

function Sample_Mondrian_Block(j, Θ, λ, δ, ζ, tree)
    E = rand(Exponential(1/Linear_dimension(Θ)))
    if j.node_type[3]==true
        τₚ = 0
    else
        τₚ = (get(j.parent)).τ
    end
    if τₚ + E < λ
        d,x = sample_split_dimension(Θ)
        j.δ = d
        j.ζ = x
        Θᴸ = copy(Θ)
        Θᴿ = copy(Θ)
        Θᴸ.Intervals[d,2]=x
        Θᴿ.Intervals[d,1]=x
        left = Mondrian_Node(τₚ+E, [true,false,false])
        left.parent = j
        left.Θ = Θᴸ
        left.tab = zeros(size(j.tab))
        left.c = zeros(size(j.tab))
        left.Gₚ = zeros(size(unique(D[2]),1))
        right = Mondrian_Node(τₚ+E, [true,false,false])
        right.parent = j
        right.Θ = Θᴿ
        right.tab = zeros(size(j.tab))
        right.c = zeros(size(j.tab))
        right.Gₚ=zeros(size(unique(D[2]),1))
        j.left = left
        j.right = right
        Sample_Mondrian_Block(left, Θᴸ, λ, δ, ζ, tree)
        Sample_Mondrian_Block(right, Θᴿ,λ, δ, ζ, tree)
    else
        j.τ = λ
        j.node_type = [false,true,false]
        push!(tree.leaves,j)
    end
end

function get_data_indices(Θ::Axis_Aligned_Box, D::Array{Float64,2})
    indices = []
    include = false
    for i in 1:size(D,1)
        for j in 1:size(Θ.Intervals,1)
            if (D[i,j] < Θ.Intervals[j,1] || D[i,j] > Θ.Intervals[j,2])
                include = false
                break
            end
            include = true
        end
        if (include)    
            push!(indices, i)
        end
    end
    return indices
end

function set_label_distribution!(j,Θ, D)
    indices = get_data_indices(Θ, D)
    for k in 1:length(unique(D[2]))
        j.Gₚ = length(D[2][D[2].==k])
    end
    j.Gₚ = j.Gₚ/(sum(j.Gₚ))
end

function initialize_posterior_leaf_counts!(Tree, D)
    X = D[1]
    Y = D[2]
    for leaf in Tree.leaves
        indices = get_data_indices(get(leaf.Θ),X)
        for k in 1:length(leaf.c)
            leaf.c[k] = length(Y[Y.==k])
            leaf.tab[k] = min(leaf.c[k],1)
        end
    end
end

function initialize_posterior_counts!(Tree,D)
    initialize_posterior_leaf_counts!(Tree,D)
    for leaf in Tree.leaves
        j = leaf
        while true
            if j.node_type[2]==false
                for k in 1:length(j.c)
                    j.c[k] = get(j.left).tab[k]+get(j.right).tab[k]
                end
            end
            for k in 1:length(j.c)
                j.tab[k] = min(j.c[k],1)
            end
            if j.node_type[3]==true
                break
            else
                j = get(j.parent)
            end
        end
    end
end

function update_posterior_counts!(leaf,y)
    leaf.c[y] += 1
    l = leaf
    while true
        if l.tab[y] == 1
            return
        else
            if l.node_type[2] == false
                l.c[y] = get(l.left).tab[y]+get(l.right).tab[y]
            l.tab[y] = min(l.c[y],1)
            end
            if l.node_type[3] == true
                return
            else
                l = get(l.parent)
            end
        end
    end         
end

function update_posterior_counts_batch!(Tree,D)
    for leaf in Tree.leaves
        indices = get_data_indices(get(leaf.Θ),D[1])
        for y in D[2][indices,:]
            update_posterior_counts(leaf,y)
        end
    end
end

function compute_predictive_posterior_distribution!(Tree, γ=1)
    J = [get(Tree.root)]
    while (size(J,1) != 0)
        j = shift!(J)
        if (j.node_type[3]==true)
            p = ones(length(j.c))./length(j.c)
            d = exp(-γ*(j.τ))
        else 
            d = exp(-γ*(j.τ-get(j.parent).τ))
            p = get(j.parent).Gₚ
        end
        for k in 1:length(j.c)
            j.Gₚ[k] = (1/(sum(j.c)+1))*(j.c[k]-d*j.tab[k]+sum(j.tab)*p[k])
        end
        if j.node_type[2] == false 
            push!(J, get(j.left))
            push!(J, get(j.right))
        end
    end
end

function expected_discount(nⱼ, Δⱼ,γ=1)
    Δ = rand(Truncated(Exponential(1/nⱼ),0,Δⱼ),1000)
    return mean(exp.(-γ*Δ))
end

function predict!(T,x, γ=1)
    j = get(T.root)
    not_sep = 1
    s = zeros(size(j.c,1))
    while true
        if (j.node_type[3] == true)
            Δⱼ = j.τ
        else
            Δⱼ = j.τ - get(j.parent).τ
        end
        nⱼ=0
        for d in size(x,1)
            nⱼ += max(x[d]-get(j.Θ).Intervals[d,2],0) + max(get(j.Θ).Intervals[d,1]-x[d],0)
        end
        pⱼ = 1-exp(Δⱼ*nⱼ)
        if pⱼ > 0
            # x branches
            jₓ = Mondrian_Node()
            if (j == get(j.parent).left)
                get(j.parent).left = jₓ
            else
                get(j.parent).right = jₓ
            end
            jₓ.parent = get(j.parent)
            j.parent = jₓ
            jₓ.left = j
            jₓ.right = Mondrian_Node()
            d = expected_discount(γ, nⱼ, Δⱼ)
            for k in 1:length(j.c)
                jₓ.c[k] = min(j.c[k],1)
            end
            jₓ.tab = jₓ.c
            for k in 1:length(jₓ.c)
                jₓ.Gₚ[k] = 1/(sum(jₓ.c)+1)*(jₓ.c[k] - d*jₓ.tab[k]+d*sum(jₓ.tab)*get(jₓ.parent).Gₚ[k])
            end
            for k in 1:length(s)
                s[k] += not_sep*(1-pⱼ)*jₓ.Gₚ[k]
            end
        end
        if j.node_type[2] == true
            for k in 1:length(s)
                s[k] += not_sep*(1-pⱼ)*j.Gₚ[k]
            end
            return s
        else
            not_sep = not_sep*(1-pⱼ)
            if x[get(j.δ)] <= get(j.ζ)
                j = get(j.left)
            else
                j = get(j.right)
            end
        end
    end
end

predict! (generic function with 2 methods)

In [175]:
function Fakedata(n,dim) 
    x = randn(n,dim)
    y = (sum(exp.(x)/(1+exp.(x)),2)).>0.5
    return 1.0*x,1*y[:,1]
end

d=4
X, Y = Fakedata(1000,d);
D = (X,Y.+1)

([0.583392 -1.3798 0.600088 -2.52453; 0.532881 0.721141 -0.0590512 1.25264; … ; -0.919538 1.67944 -1.10655 -1.01816; 0.688278 -0.0430784 0.25427 2.49178], [1, 2, 1, 1, 2, 1, 1, 2, 2, 2  …  2, 1, 2, 1, 2, 2, 1, 2, 2, 2])

In [186]:
MT = Sample_Mondrian_Tree(0.1,D);
initialize_posterior_counts!(MT,D);
compute_predictive_posterior_distribution!(MT,10*size(D[1],2));

In [187]:
using MLBase
pred = []
for i in 1:size(D[1],1) 
    p = predict(MT,D[1][i,:],10*size(D[1],2))
    if p[1] > p[2]
        push!(pred,1)
    else
        push!(pred,2)
    end
end
println(correctrate(D[2],convert(Array{Int,1},pred)))
println(unique(pred))

0.573
[1]


In [137]:
for leaf in MT.leaves
    println(leaf.Gₚ)
end

[0.559435, 0.441553]
[0.559435, 0.441553]
[0.559256, 0.441374]
[0.559256, 0.441374]
[0.559496, 0.441614]
[0.559496, 0.441614]
[0.55895, 0.441068]
[0.5595, 0.441618]
[0.5595, 0.441618]
[0.559232, 0.44135]
[0.559392, 0.44151]
[0.559392, 0.44151]
[0.55908, 0.441198]
[0.55908, 0.441198]
[0.558581, 0.440699]
[0.558581, 0.440699]
[0.55908, 0.441198]
[0.559156, 0.441274]
[0.559156, 0.441274]
[0.559285, 0.441403]
[0.559271, 0.441389]
[0.559271, 0.441389]
[0.559271, 0.441389]
[0.559271, 0.441389]
[0.55888, 0.440998]
[0.554577, 0.436695]
[0.554577, 0.436695]
[0.554577, 0.436695]
[0.556592, 0.43871]
[0.556592, 0.43871]
[0.552468, 0.434586]
[0.559409, 0.441527]
[0.5592, 0.441318]
[0.558767, 0.440884]
[0.558767, 0.440884]
[0.559028, 0.441146]
[0.559378, 0.441496]
[0.559403, 0.441521]
[0.559117, 0.441235]
[0.559212, 0.44133]
[0.559212, 0.44133]
[0.559212, 0.44133]
[0.553845, 0.435963]
[0.553845, 0.435963]
[0.553845, 0.435963]
[0.553845, 0.435963]
[0.559077, 0.441195]
[0.559383, 0.441501]
[0.559359, 

In [138]:
include("load_titanic.jl")
x_train, y_train, x_test, y_test = load();
D = (x_train, y_train.+1)
MT = Sample_Mondrian_Tree(0.0005,D);
initialize_posterior_counts(MT,D);
compute_predictive_posterior_distribution(MT,1);

Any[1, 3, 6, 7, 8, 9, 11, 12, 14, 15, 18, 19, 20, 22, 25, 26, 27, 28, 30, 31, 32, 33, 34, 35, 37, 39, 40, 41, 42, 43, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 68, 71, 74, 75, 76, 78, 79, 82, 83, 85, 86, 87, 88, 89, 90, 92, 93, 94, 95, 96, 97, 98, 100, 101, 102, 103, 104, 105, 106, 107, 113, 114, 117, 118, 119, 121, 123, 124, 126, 128, 130, 131, 132, 135, 136, 137, 141, 142, 145, 146, 147, 148, 149, 151, 153, 155, 156, 157, 159, 160, 161, 162, 164, 166, 167, 169, 170, 171, 172, 173, 174, 176, 177, 180, 183, 184, 190, 193, 196, 199, 200, 201, 202, 203, 205, 206, 207, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 225, 227, 228, 229, 230, 234, 235, 238, 239, 243, 244, 245, 246, 248, 249, 250, 251, 252, 253, 256, 259, 261, 262, 263, 264, 265, 266, 267, 268, 269, 271, 272, 274, 276, 278, 279, 280, 281, 282, 283, 285, 286, 288, 289, 290, 291, 294, 295, 296, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 312, 

In [140]:
using MLBase
pred = []
for i in 1:size(D[1],1) 
    p = predict(MT,D[1][i,:],10*size(D[1],2))
    if p[1] > p[2]
        push!(pred,1)
    else
        push!(pred,2)
    end
end
correctrate(D[2],convert(Array{Int,1},pred))
#pred

0.5914826498422713

In [7]:
compute_predictive_posterior_distribution(MT,1);

2
2
2
2
0
0
2
2
0
0
0
0
0
0
2
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
2
0
0
0
0
0
0
1000
1000
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1000
1000
1000
1000
0
0
4
4
0
0
0
0
1000
1000
0
0
0
0
0
0
0
0
0
0
1000
1000
0
0
0
0
0
0
0
0
0
0
0
0
1000
1000
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
0
0
0
0
0
0
1000
1000
1000
1000
1000
1000
1000
1000
0
0
1000
1000
0
0
1000
1000
1000
1000
0
0
0
0
0
0
0
0
1000
1000
1000
1000
1000
1000
0
0
0
0
0
0
0
0
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
0
0
0
0
0
0
1000
1000
0
0
1000
1000
0
0
1000
1000
1000
1000
0
0
0
0
0
0
0
0
1000
1000
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1000
1000
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1000
1000
0
0
1000
1000
0
0
0
0
0
0
0
0
1000
1000
0
0
0
0
0
0
1000
1000
0
0
1000
1000
1000
1000
0
0
1000
1000
0
0
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
0
0
0
0
1000
1000
0
0
0
0
1000
1000
0
0
1000
1000
1000
1000
1000
1000
1000
100

In [287]:
a.Intervals

LoadError: [91mtype Mondrian_Node has no field Intervals[39m

In [43]:
D[1][1,2] < a.Intervals[2,1] || D[1][1,2] > a.Intervals[2,2]

false

In [40]:
D[1,2]

LoadError: [91mMethodError: no method matching getindex(::Tuple{Array{Float64,2},Array{Int64,1}}, ::Int64, ::Int64)[39m

In [22]:
Y = [1,2,2,2,1]

5-element Array{Int64,1}:
 1
 2
 2
 2
 1

In [24]:
size(Y[Y.==1],1)

2

In [16]:
ones(2)

2-element Array{Float64,1}:
 1.0
 1.0

In [17]:
exp(2)

7.38905609893065

In [19]:
sum([1,2,3])

6